In [1]:
import pandas as pd
import pickle

In [2]:
PATH = "./"

In [3]:
with open('trained_models/task1_RFmodel.pkl', 'rb') as file:
    task1 = pickle.load(file)
    
with open('trained_models/task2_OCSVMmodel.pkl', 'rb') as file:
    task21 = pickle.load(file)

with open('trained_models/task2_RFmodel.pkl', 'rb') as file:
    task2 = pickle.load(file)

with open('trained_models/task3_RFmodel.pkl', 'rb') as file:
    task3 = pickle.load(file)
    
with open('trained_models/task4_RFmodel.pkl', 'rb') as file:
    task4 = pickle.load(file)

In [4]:
print("Models loaded: ", task1, task21, task2, task3, task4)

Models loaded:  RandomForestClassifier(random_state=42) OneClassSVM() RandomForestClassifier() RandomForestClassifier() RandomForestClassifier()


In [5]:
#Creazione dati da classificare

# Caricare e concatenare i file CSV (Case178.csv-Case223.csv)
data_frames = []
for i in range(178, 224):
    file_path = (PATH + f'/dataset/test/data/Case{i:03d}.csv')
    df = pd.read_csv(file_path)
    df['Case'] = i
    data_frames.append(df)

prediction_data = pd.concat(data_frames, ignore_index=True)
prediction_data

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
55241,1.196,2.017816,2.016297,2.012680,2.044149,2.037457,2.033628,2.069056,223
55242,1.197,2.039878,2.055107,2.054626,2.071298,2.072899,2.076782,2.093548,223
55243,1.198,2.054024,2.081678,2.094493,2.096693,2.095300,2.107353,2.138567,223
55244,1.199,2.057180,2.094954,2.097106,2.153193,2.099423,2.112326,2.127471,223


# TASK 1

In [6]:
# Preparazione dei dati di test
features = prediction_data[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data[["Case"]]

# Predizione con il modello addestrato
predictions = task1.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

df1 = pd.read_csv(PATH + '/dataset/answer.csv')
df1 = df1[['ID', 'task1']]
df1.columns = ['Case', 'Real'] # Rinomina le colonne

df2 = prediction_result

# Effettua il join tra i due dataframe
merged_df = pd.merge(df2, df1, on='Case', how='left')

# Stampa il nuovo dataframe
merged_df

/var/folders/k7/8yszcg2s7nq1zy9gv8rzd3jc0000gn/T/ipykernel_50438/2600455823.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


,Case,Prediction,Real
0,178,1,1
1,179,1,1
2,180,0,0
3,181,0,1
4,182,0,0
5,183,0,0
6,184,1,1
7,185,0,0
8,186,1,1
9,187,0,0


In [7]:
# Calcola il numero totale di righe
total_rows = len(merged_df)

# Calcola il numero di righe con predizioni errate
wrong_predictions = merged_df[merged_df['Prediction'] != merged_df['Real']]
num_wrong = len(wrong_predictions)

# Calcola la percentuale di dati sbagliati
percentage_wrong = (num_wrong / total_rows) * 100

print(f"Numero totale di righe: {total_rows}")
print(f"Numero di righe con predizioni errate: {num_wrong}")
print(f"Percentuale di dati sbagliati: {percentage_wrong:.2f}%")

Numero totale di righe: 46
Numero di righe con predizioni errate: 8
Percentuale di dati sbagliati: 17.39%


# TASK 2

In [8]:
cases_to_keep = merged_df.loc[merged_df['Prediction'] == 1, 'Case']

# Crea prediction_data2
prediction_data2 = prediction_data.loc[prediction_data['Case'].isin(cases_to_keep)]

prediction_data2

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
54040,1.196,2.999206,2.985047,2.982110,2.970652,3.004048,3.009258,3.006424,222
54041,1.197,2.977775,2.952302,2.945279,2.933262,2.965413,2.964095,2.962946,222
54042,1.198,2.959111,2.930013,2.915038,2.926461,2.932936,2.921135,2.922863,222
54043,1.199,2.948195,2.921907,2.911228,2.918398,2.910006,2.895828,2.893770,222


In [9]:
prediction_data2["Case"].unique()

array([178, 179, 184, 186, 188, 190, 192, 193, 196, 197, 200, 201, 202,
       203, 204, 205, 206, 207, 208, 209, 211, 212, 214, 215, 216, 218,
       219, 220, 221, 222])

In [10]:
# Preparazione dei dati di test
features = prediction_data2[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data2[["Case"]]

# Predizione con il modello addestrato
predictions = task21.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

/var/folders/k7/8yszcg2s7nq1zy9gv8rzd3jc0000gn/T/ipykernel_50438/2018682388.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


In [11]:
df1 = pd.read_csv(PATH + '/dataset/answer.csv')
df1 = df1[['ID', 'task2']]
df1.columns = ['Case', 'Real'] # Rinomina le colonne

df2 = prediction_result

# Effettua il join tra i due dataframe
merged_df = pd.merge(df2, df1, on='Case', how='left')

# Stampa il nuovo dataframe
merged_df

,Case,Prediction,Real
0,178,1,2
1,179,1,3
2,184,-1,1
3,186,1,2
4,188,1,3
5,190,1,3
6,192,-1,1
7,193,1,2
8,196,1,2
9,197,1,2


219 no

In [12]:
cases_to_keep = merged_df.loc[merged_df['Prediction'] == 1, 'Case']

# Crea prediction_data2
prediction_data2 = prediction_data.loc[prediction_data['Case'].isin(cases_to_keep)]

prediction_data2

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
52839,1.196,2.047358,2.106033,2.116920,2.169856,2.066395,2.086070,2.086723,221
52840,1.197,2.062200,2.104518,2.115623,2.195780,2.089076,2.099911,2.095505,221
52841,1.198,2.062772,2.093498,2.096172,2.163714,2.091489,2.115760,2.108553,221
52842,1.199,2.045088,2.077390,2.083716,2.120518,2.088176,2.111408,2.103875,221


In [13]:
# Preparazione dei dati di test
features = prediction_data2[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data2[["Case"]]

# Predizione con il modello addestrato
predictions = task2.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

df1 = pd.read_csv(PATH + '/dataset/answer.csv')
df1 = df1[['ID', 'task2']]
df1.columns = ['Case', 'Real'] # Rinomina le colonne
df1['Real'] = df1['Real'].replace({2: 1, 3: 2})

df2 = prediction_result

# Effettua il join tra i due dataframe
merged_df = pd.merge(df2, df1, on='Case', how='left')

# Stampa il nuovo dataframe
merged_df

/var/folders/k7/8yszcg2s7nq1zy9gv8rzd3jc0000gn/T/ipykernel_50438/2948642296.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


,Case,Prediction,Real
0,178,1,1
1,179,2,2
2,186,1,1
3,188,2,2
4,190,2,2
5,193,1,1
6,196,1,1
7,197,1,1
8,201,2,0
9,202,2,2


In [14]:
# Calcola il numero totale di righe
total_rows = len(merged_df)

# Calcola il numero di righe con predizioni errate
wrong_predictions = merged_df[merged_df['Prediction'] != merged_df['Real']]
num_wrong = len(wrong_predictions)

# Calcola la percentuale di dati sbagliati
percentage_wrong = (num_wrong / total_rows) * 100

print(f"Numero totale di righe: {total_rows}")
print(f"Numero di righe con predizioni errate: {num_wrong}")
print(f"Percentuale di dati sbagliati: {percentage_wrong:.2f}%")

Numero totale di righe: 23
Numero di righe con predizioni errate: 7
Percentuale di dati sbagliati: 30.43%


# TASK 3

In [15]:
cases_to_keep = merged_df.loc[merged_df['Prediction'] == 1, 'Case']

prediction_data3 = prediction_data.loc[prediction_data['Case'].isin(cases_to_keep)]

prediction_data3

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
52839,1.196,2.047358,2.106033,2.116920,2.169856,2.066395,2.086070,2.086723,221
52840,1.197,2.062200,2.104518,2.115623,2.195780,2.089076,2.099911,2.095505,221
52841,1.198,2.062772,2.093498,2.096172,2.163714,2.091489,2.115760,2.108553,221
52842,1.199,2.045088,2.077390,2.083716,2.120518,2.088176,2.111408,2.103875,221


In [16]:
prediction_data3["Case"].unique()

array([178, 186, 193, 196, 197, 204, 209, 221])

In [17]:
# Preparazione dei dati di test
features = prediction_data3[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data3[["Case"]]

# Predizione con il modello addestrato
predictions = task3.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

df1 = pd.read_csv(PATH + '/dataset/answer.csv')
df1 = df1[['ID', 'task3']]
df1.columns = ['Case', 'Real'] # Rinomina le colonne

df2 = prediction_result

# Effettua il join tra i due dataframe
merged_df3 = pd.merge(df2, df1, on='Case', how='left')

# Stampa il nuovo dataframe
merged_df3

/var/folders/k7/8yszcg2s7nq1zy9gv8rzd3jc0000gn/T/ipykernel_50438/2846478374.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


,Case,Prediction,Real
0,178,2,2
1,186,6,6
2,193,1,1
3,196,4,4
4,197,7,7
5,204,3,3
6,209,7,7
7,221,2,2


In [20]:
# Calcola il numero totale di righe
total_rows = len(merged_df3)

# Calcola il numero di righe con predizioni errate
wrong_predictions = merged_df3[merged_df3['Prediction'] != merged_df3['Real']]
num_wrong = len(wrong_predictions)

# Calcola la percentuale di dati sbagliati
percentage_wrong = (num_wrong / total_rows) * 100

print(f"Numero totale di righe: {total_rows}")
print(f"Numero di righe con predizioni errate: {num_wrong}")
print(f"Percentuale di dati sbagliati: {percentage_wrong:.2f}%")

Numero totale di righe: 8
Numero di righe con predizioni errate: 0
Percentuale di dati sbagliati: 0.00%


# TASK 4

In [21]:
cases_to_keep = merged_df.loc[merged_df['Prediction'] == 2, 'Case']

prediction_data4 = prediction_data.loc[prediction_data['Case'].isin(cases_to_keep)]

prediction_data4

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
1201,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1202,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1203,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1204,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1205,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
...,...,...,...,...,...,...,...,...,...
51638,1.196,2.016943,2.015412,2.010793,2.027649,2.036357,2.049105,2.027695,220
51639,1.197,2.039344,2.053950,2.052734,2.046453,2.072185,2.102077,2.066795,220
51640,1.198,2.053662,2.080897,2.095161,2.060417,2.095059,2.122962,2.086246,220
51641,1.199,2.057250,2.094755,2.096188,2.109464,2.099592,2.128246,2.097720,220


In [22]:
prediction_data4["Case"].unique()

array([179, 188, 190, 201, 202, 203, 205, 206, 208, 211, 212, 214, 215,
       216, 220])

In [23]:
# Preparazione dei dati di test
features = prediction_data3[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data3[["Case"]]

# Predizione con il modello addestrato
predictions = task4.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

df1 = pd.read_csv(PATH + '/dataset/answer.csv')
df1 = df1[['ID', 'task4']]
df1.columns = ['Case', 'Real'] # Rinomina le colonne

df2 = prediction_result

# Effettua il join tra i due dataframe
merged_df4 = pd.merge(df2, df1, on='Case', how='left')

# Stampa il nuovo dataframe
merged_df4

/var/folders/k7/8yszcg2s7nq1zy9gv8rzd3jc0000gn/T/ipykernel_50438/3596217183.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


,Case,Prediction,Real
0,178,2,0
1,186,3,0
2,193,2,0
3,196,2,0
4,197,4,0
5,204,1,0
6,209,4,0
7,221,2,0
